In [1]:
# deletes engine - run when you have to restart
!rm world_dev.sqlite

In [2]:
# sqlalchemy dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
from sqlalchemy import func

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Text, Numeric

import os

In [3]:
# pandas dependencies
import pandas as pd
import pandas_profiling

# display multiple print results on one line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# flask 
from flask import Flask, jsonify

In [6]:
# read csv
df = pd.read_csv('data3.csv')
df.fillna(0)
df.info()
df.head()

,Unnamed: 0,IsoCode,CountryName,Region,IncomeGroup,GiniYear,GiniCoefficient,HomicideYear,HomicidePer100k,HDI,GDP_Mean,LifeExpectancy,services2015,industry2015,agriculture2015,GDP_PPP_Year,GDP_PPP,Unemployment2015,Unemployment2016,Unemployment2017
0,0,AFG,Afghanistan,South Asia,Low income,0.0,0.0,2012.0,6.60,0.48,5.61,63.30,55.65,22.95,21.40,2016.0,1944.12,8.6,8.6,8.5
1,1,ALB,Albania,Europe & Central Asia,Upper middle income,2012.0,29.0,2015.0,2.30,0.76,2.29,78.20,52.96,24.18,22.86,2016.0,11540.03,17.1,15.2,15.0
2,2,DZA,Algeria,Middle East & North Africa,Upper middle income,2011.0,27.6,2015.0,1.40,0.75,3.36,75.86,48.59,38.81,12.61,2016.0,15013.30,11.2,11.5,11.7
3,3,AGO,Angola,Sub-Saharan Africa,Lower middle income,2008.0,42.7,2015.0,9.60,0.53,3.78,61.19,0.00,0.00,0.00,2016.0,6454.14,6.2,6.1,6.2
4,4,ATA,Antarctica,0,0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0
5,5,ARG,Argentina,Latin America & Caribbean,Upper middle income,2014.0,42.7,2015.0,6.50,0.83,2.20,76.29,65.89,28.06,6.04,2016.0,19939.93,7.2,8.8,8.1
6,6,ARM,Armenia,Europe & Central Asia,Lower middle income,2015.0,32.4,2015.0,2.50,0.74,3.49,74.21,51.97,28.76,19.27,2016.0,8832.76,18.3,17.6,17.4
7,7,AUS,Australia,East Asia & Pacific,High income,2010.0,34.7,2015.0,1.00,0.94,2.63,82.45,72.04,25.40,2.56,2016.0,46012.33,6.1,5.7,5.7
8,8,AUT,Austria,Europe & Central Asia,High income,2014.0,30.5,2015.0,0.50,0.89,1.24,81.84,70.70,28.06,1.25,2016.0,50551.55,5.7,6.0,5.7
9,9,AZE,Azerbaijan,Europe & Central Asia,Upper middle income,2008.0,31.8,2014.0,2.50,0.76,1.85,71.85,43.92,49.30,6.78,2016.0,17256.63,5.0,5.0,5.1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 20 columns):
Unnamed: 0          177 non-null int64
IsoCode             177 non-null object
CountryName         177 non-null object
Region              172 non-null object
IncomeGroup         172 non-null object
GiniYear            172 non-null float64
GiniCoefficient     172 non-null float64
HomicideYear        172 non-null float64
HomicidePer100k     172 non-null float64
HDI                 166 non-null float64
GDP_Mean            171 non-null float64
LifeExpectancy      170 non-null float64
services2015        150 non-null float64
industry2015        150 non-null float64
agriculture2015     150 non-null float64
GDP_PPP_Year        165 non-null float64
GDP_PPP             165 non-null float64
Unemployment2015    169 non-null float64
Unemployment2016    169 non-null float64
Unemployment2017    169 non-null float64
dtypes: float64(15), int64(1), object(4)
memory usage: 27.7+ KB


,Unnamed: 0,IsoCode,CountryName,Region,IncomeGroup,GiniYear,GiniCoefficient,HomicideYear,HomicidePer100k,HDI,GDP_Mean,LifeExpectancy,services2015,industry2015,agriculture2015,GDP_PPP_Year,GDP_PPP,Unemployment2015,Unemployment2016,Unemployment2017
0,0,AFG,Afghanistan,South Asia,Low income,0.0,0.0,2012.0,6.6,0.48,5.61,63.30,55.65,22.95,21.40,2016.0,1944.12,8.6,8.6,8.5
1,1,ALB,Albania,Europe & Central Asia,Upper middle income,2012.0,29.0,2015.0,2.3,0.76,2.29,78.20,52.96,24.18,22.86,2016.0,11540.03,17.1,15.2,15.0
2,2,DZA,Algeria,Middle East & North Africa,Upper middle income,2011.0,27.6,2015.0,1.4,0.75,3.36,75.86,48.59,38.81,12.61,2016.0,15013.30,11.2,11.5,11.7
3,3,AGO,Angola,Sub-Saharan Africa,Lower middle income,2008.0,42.7,2015.0,9.6,0.53,3.78,61.19,NaN,NaN,NaN,2016.0,6454.14,6.2,6.1,6.2
4,4,ATA,Antarctica,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
updated_df = df.rename(columns={"Unnamed: 0": "id"
                  })
updated_df.set_index("id").head()
updated_df.shape

,IsoCode,CountryName,Region,IncomeGroup,GiniYear,GiniCoefficient,HomicideYear,HomicidePer100k,HDI,GDP_Mean,LifeExpectancy,services2015,industry2015,agriculture2015,GDP_PPP_Year,GDP_PPP,Unemployment2015,Unemployment2016,Unemployment2017
id,,,,,,,,,,,,,,,,,,,
0,AFG,Afghanistan,South Asia,Low income,0.0,0.0,2012.0,6.6,0.48,5.61,63.30,55.65,22.95,21.40,2016.0,1944.12,8.6,8.6,8.5
1,ALB,Albania,Europe & Central Asia,Upper middle income,2012.0,29.0,2015.0,2.3,0.76,2.29,78.20,52.96,24.18,22.86,2016.0,11540.03,17.1,15.2,15.0
2,DZA,Algeria,Middle East & North Africa,Upper middle income,2011.0,27.6,2015.0,1.4,0.75,3.36,75.86,48.59,38.81,12.61,2016.0,15013.30,11.2,11.5,11.7
3,AGO,Angola,Sub-Saharan Africa,Lower middle income,2008.0,42.7,2015.0,9.6,0.53,3.78,61.19,NaN,NaN,NaN,2016.0,6454.14,6.2,6.1,6.2
4,ATA,Antarctica,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(177, 20)

In [8]:
# pandas profile report
# pandas_profiling.ProfileReport(updated_df)

In [9]:
# create sqlite database
engine = create_engine("sqlite:///world_dev.sqlite")
conn = engine.connect()

In [10]:
# declarative_base
# Base = declarative_base()
Base = automap_base()

In [11]:
# create world_dev metadata class
class world_dev(Base):
    __tablename__ = 'world_dev'
    
    id = Column(Integer, primary_key=True)
    IsoCode = Column(Text(64))
    CountryName = Column(Text(64))
    Region = Column(Text(64))
    IncomeGroup = Column(Text(64))
    GiniYear = Column(Float(64))
    GiniCoefficient = Column(Float(64))
    HomicideYear = Column(Float(64))
    HomicidePer100k = Column(Float(64))
    HDI = Column(Float(64))
    GDP_Mean = Column(Float(64))
    LifeExpectancy = Column(Float(64))
    services2015 = Column(Float(64))
    industry2015 = Column(Float(64))
    agriculture2015 = Column(Float(64))
    GDP_PPP_Year = Column(Float(64))
    GDP_PPP = Column(Float(64))
    Unemployment2015 = Column(Float(64))
    Unemployment2016 = Column(Float(64))
    Unemployment2017 = Column(Float(64))
    
    
    def __repr__(self):
        return f"id={self.ID}"

In [12]:
# create a "metadata" layer that abstracts our SQL Database
Base.metadata.create_all(engine)
engine.table_names()

['world_dev']

In [13]:
# create a session object to connect to DB
session = Session(bind=engine)

In [14]:
# add updated_df csv data
updated_df.to_sql('world_dev', conn, if_exists='append', index=False)

In [15]:
# read back world_dev class to see if updated_df data was appended to world_dev.sqlite
data_world_dev = pd.read_sql("SELECT * FROM world_dev", conn)
# data_world_dev.head()
# data_world_dev.tail()
data_world_dev.shape

(177, 20)

In [16]:
# store data_world_dev into new variable and set index to id
final_sql_df = data_world_dev.set_index('id')
final_sql_df.head()

,IsoCode,CountryName,Region,IncomeGroup,GiniYear,GiniCoefficient,HomicideYear,HomicidePer100k,HDI,GDP_Mean,LifeExpectancy,services2015,industry2015,agriculture2015,GDP_PPP_Year,GDP_PPP,Unemployment2015,Unemployment2016,Unemployment2017
id,,,,,,,,,,,,,,,,,,,
0,AFG,Afghanistan,South Asia,Low income,0.0,0.0,2012.0,6.6,0.48,5.61,63.30,55.65,22.95,21.40,2016.0,1944.12,8.6,8.6,8.5
1,ALB,Albania,Europe & Central Asia,Upper middle income,2012.0,29.0,2015.0,2.3,0.76,2.29,78.20,52.96,24.18,22.86,2016.0,11540.03,17.1,15.2,15.0
2,DZA,Algeria,Middle East & North Africa,Upper middle income,2011.0,27.6,2015.0,1.4,0.75,3.36,75.86,48.59,38.81,12.61,2016.0,15013.30,11.2,11.5,11.7
3,AGO,Angola,Sub-Saharan Africa,Lower middle income,2008.0,42.7,2015.0,9.6,0.53,3.78,61.19,NaN,NaN,NaN,2016.0,6454.14,6.2,6.1,6.2
4,ATA,Antarctica,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# save to csv

final_sql_df.to_csv('worldDev.csv')

# this sheet stores the avg gini inserted in place of null
# final_sql_df.to_csv('worldDev2.csv')

In [18]:
# create object for worldDev.csv
worldDevcsv = 'worldDev.csv'

# using the worldDev2csv that replaced null gini with the avg gini
# worldDevcsv = 'worldDev2.csv'


In [19]:
# function to populate table
def populate_table (engine, table, csvfile):
    """Populates a table from a Pandas DataFrame."""
    
    # connect to the database
    conn = engine.connect()
    
    # Load the CSV file into a pandas dataframe
    insert_df = pd.read_csv(csvfile)
    
    # Orient='records' creates a list of data to write
    # http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
    data = insert_df.to_dict(orient='records')
    print(data[:1])
    
    # Optional: Delete all rows in the table 
    conn.execute(table.delete())

    # Insert the dataframe into the database in one bulk insert
    conn.execute(table.insert(), data)



In [20]:
# Call the function to insert the data for each table
# world_dev is the class
populate_table(engine, world_dev.__table__, worldDevcsv)

[{'id': 0, 'IsoCode': 'AFG', 'CountryName': 'Afghanistan', 'Region': 'South Asia', 'IncomeGroup': 'Low income', 'GiniYear': 0.0, 'GiniCoefficient': 0.0, 'HomicideYear': 2012.0, 'HomicidePer100k': 6.6, 'HDI': 0.48, 'GDP_Mean': 5.61, 'LifeExpectancy': 63.3, 'services2015': 55.65, 'industry2015': 22.95, 'agriculture2015': 21.4, 'GDP_PPP_Year': 2016.0, 'GDP_PPP': 1944.12, 'Unemployment2015': 8.6, 'Unemployment2016': 8.6, 'Unemployment2017': 8.5}]


In [21]:
# check to see if data was added to table
engine.execute("SELECT * FROM world_dev LIMIT 1").fetchall()

[(0, 'AFG', 'Afghanistan', 'South Asia', 'Low income', 0.0, 0.0, 2012.0, 6.6, 0.48, 5.61, 63.3, 55.65, 22.95, 21.4, 2016.0, 1944.12, 8.6, 8.6, 8.5)]

### End Database Engineering Section
<br>
<hr>

### Reflect Table Section
<hr>

In [22]:
# Use SQLAlchemy automap_base() to refelct tables into classes and save a reference to those classes
Base = automap_base()

In [23]:
# use the base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [24]:
# print all of the classes mapped to the Base
Base.classes.keys()

['world_dev']

In [25]:
# assign worldDev class to variables
WorldDev = Base.classes.world_dev

In [26]:
# create a session
session = Session(engine)

In [27]:
# Display the row's columns and data in dict format
results = session.query(WorldDev.CountryName, WorldDev.GiniCoefficient).all()

In [28]:
# Prints Country Name and GiniCoefficient
results

[('Afghanistan', 0.0),
 ('Albania', 29.0),
 ('Algeria', 27.6),
 ('Angola', 42.7),
 ('Antarctica', None),
 ('Argentina', 42.7),
 ('Armenia', 32.4),
 ('Australia', 34.7),
 ('Austria', 30.5),
 ('Azerbaijan', 31.8),
 ('Bahamas', 0.0),
 ('Bangladesh', 32.1),
 ('Belarus', 26.7),
 ('Belgium', 28.1),
 ('Belize', 0.0),
 ('Benin', 47.8),
 ('Bermuda', 0.0),
 ('Bhutan', 38.8),
 ('Bolivia', 45.8),
 ('Bosnia and Herzegovina', 33.8),
 ('Botswana', 60.5),
 ('Brazil', 51.3),
 ('Brunei Darussalam', 0.0),
 ('Bulgaria', 37.4),
 ('Burkina Faso', 35.3),
 ('Burundi', 39.2),
 ('Cambodia', 0.0),
 ('Cameroon', 46.5),
 ('Canada', 34.0),
 ('Central African Republic', 56.2),
 ('Chad', 43.3),
 ('Chile', 47.7),
 ('China', 42.2),
 ('Colombia', 51.1),
 ('Congo�(Brazzaville)', 48.9),
 ('Congo, (Kinshasa)', 42.1),
 ('Costa Rica', 48.2),
 ("C�te d'Ivoire", 41.7),
 ('Croatia', 32.2),
 ('Cuba', 0.0),
 ('Cyprus', 35.6),
 ('Czech Republic', 25.9),
 ('Denmark', 28.5),
 ('Djibouti', 44.1),
 ('Dominican Republic', 44.9),
 ('Ecu